In [1]:
import pandas as pd
import numpy as np
import janitor as jn
from janitor import Inequality_Condition
from janitor.utils import _generic_less_than_inequality, _generic_greater_than_inequality,_conditional_merge_compute
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_string_dtype
from io import StringIO

In [2]:
df1 = pd.DataFrame(
    [
        {"x": "b", "y": 1, "v": 1},
        {"x": "b", "y": 3, "v": 2},
        {"x": "b", "y": 6, "v": 3},
        {"x": "a", "y": 1, "v": 4},
        {"x": "a", "y": 3, "v": 5},
        {"x": "a", "y": 6, "v": 6},
        {"x": "c", "y": 1, "v": 7},
        {"x": "c", "y": 3, "v": 8},
        {"x": "c", "y": 6, "v": 9},
        {"x": "c", "y": np.nan, "v": 9},
    ]
)

df2 = pd.DataFrame(
    [{"x": "c", "v": 8, "foo": 4}, {"x": "b", "v": 7, "foo": 2}, {"x": "b", "v": 7, "foo": None}]
)

result = jn.cond_merge(df1, df2, ('y', 'foo', 'le'))

result

,x_x,y,v_x,x_y,v_y,foo
0,b,1.0,1,b,7,2.0
1,b,1.0,1,c,8,4.0
2,b,3.0,2,c,8,4.0
3,a,1.0,4,b,7,2.0
4,a,1.0,4,c,8,4.0
5,a,3.0,5,c,8,4.0
6,c,1.0,7,b,7,2.0
7,c,1.0,7,c,8,4.0
8,c,3.0,8,c,8,4.0


In [3]:
# df1.cond_merge(df2, ('y','foo','lt'), ('v','v', 'le'))

In [4]:
%load_ext line_profiler
%load_ext memory_profiler

In [5]:
df1 = pd.concat([df1]*1_000, ignore_index=True)
df2 = pd.concat([df2]*200, ignore_index = True)

In [19]:
%timeit df1.cond_merge(df2, ('y','foo','lt'), ('v','v', 'le'))

In [18]:
%lprun -f  _generic_less_than_inequality   _generic_less_than_inequality(        df=df1, right=df2, left_on="v", right_on="v"                                             , strict=True)

Timer unit: 1e-06 s

Total time: 0.244663 s
File: /workspaces/pyjanitor/janitor/utils.py
Function: _generic_less_than_inequality at line 2384

Line #      Hits         Time  Per Hit   % Time  Line Contents
  2384                                           def _generic_less_than_inequality(
  2385                                               df: pd.DataFrame,
  2386                                               right: pd.DataFrame,
  2387                                               left_on: str,
  2388                                               right_on: str,
  2389                                               strict: bool = False,
  2390                                           ):
  2391                                               """
  2392                                               Generic implementation for less than joins.
  2393                                               This covers less than or equal joins as well.
  2394                                            

In [13]:
%lprun -f _conditional_merge_compute _conditional_merge_compute(df1, df2, [Inequality_Condition('y', 'foo', 'lt')], "inner", ("_x", "_y"))

Timer unit: 1e-06 s

Total time: 0.398912 s
File: /workspaces/pyjanitor/janitor/utils.py
Function: _conditional_merge_compute at line 2512

Line #      Hits         Time  Per Hit   % Time  Line Contents
  2512                                           def _conditional_merge_compute(
  2513                                               df: pd.DataFrame,
  2514                                               right: Union[pd.DataFrame, pd.Series],
  2515                                               inequality_conditions: list,
  2516                                               how: str,
  2517                                               suffixes: tuple,
  2518                                           ):
  2519                                               """
  2520                                               Compute dataframe where how = "inner"
  2521                                               """
  2522                                           
  2523         1          3.0  

In [9]:
%lprun -f janitor.le_join janitor.le_join(df1, df2, 'y', 'foo')

UsageError: Could not find function 'janitor.le_join'.
NameError: name 'janitor' is not defined


In [10]:
df, right, left_c, right_c = _generic_greater_than_inequality(
        df=df1, right=df2, left_on="v", right_on="v"
                                             )

ValueError: not enough values to unpack (expected 4, got 2)

In [ ]:
%lprun -f  _generic_greater_than_inequality   _generic_greater_than_inequality(        df=df1, right=df2, left_on="v", right_on="v"                                             )

In [ ]:
%timeit df1.merge(df2, how='left', left_on='v', right_on='v')

In [ ]:
df1.merge(df2, how='inner', left_on='v', right_on='v')

In [ ]:
A = pd.DataFrame(dict(
        A_id=range(10),
        A_value=range(5, 105, 10)
    ))
B = pd.DataFrame(dict(
        B_id=range(5),
        B_low=[0, 30, 30, 46, 84],
        B_high=[10, 40, 50, 54, 84]
    ))

In [ ]:
A

In [ ]:
B

In [ ]:
outcome = A.le_join(B, left_on='A_value', right_on='B_high').query('A_value >= B_low')

outcome

In [ ]:
A.merge(outcome, how = 'left')

In [ ]:
df_a = pd.DataFrame([{"a": 1, "b": 4}, {"a": 2, "b": 5}, {"a": 3, "b": 6}])
df_b = pd.DataFrame([{"c": 2, "d": 7}, {"c": 3, "d": 8}])

In [ ]:
df_a

In [ ]:
df_b

In [ ]:
df_a.merge(df_a.lt_join(df_b, "a", "c"), how = 'left')

In [ ]:
df1 = pd.DataFrame({'id': [1,1,1,2,2,3], 'value_1': [2,5,7,1,3,4]})
df2 = pd.DataFrame({'id': [1,1,1,1,2,2,2,3], 'value_2A': [0,3,7,12,0,2,3,1], 'value_2B': [1,5,9,15,1,4,6,3]})

In [ ]:
df1

In [ ]:
df2

In [ ]:
df1.merge(df1.merge(df2).query('value_2A<=value_1<=value_2B'), how = 'left')

In [ ]:
data = """    company       date  measure
0         0  2010-01-01       10
1         0  2010-01-15       10
2         0  2010-02-01       10
3         0  2010-02-15       10
4         0  2010-03-01       10
5         0  2010-03-15       10
6         0  2010-04-01       10
7         1  2010-03-01        5
8         1  2010-03-15        5
9         1  2010-04-01        5
10        1  2010-04-15        5
11        1  2010-05-01        5
12        1  2010-05-15        5"""

left = pd.read_csv(StringIO(data), sep="\s{2,}", engine='python')

left['date'] = pd.to_datetime(left['date'])
left

In [ ]:
data = """   company   end_date
0        0  2010-02-01
1        0  2010-03-15
2        1  2010-04-01
3        1  2010-05-15
"""

right = pd.read_csv(StringIO(data), sep="\s{2,}", engine='python')

right['end_date'] = pd.to_datetime(right['end_date'])
right

In [ ]:
left.dtypes

In [ ]:
right.dtypes

In [ ]:
is_numeric_dtype(left.SALARY, right.grad)

In [ ]:
left.le_join(right, 'date', 'end_date', suffixes = (None, '_y')).loc[lambda df: (df['date'] >= (df['end_date']-pd.Timedelta('30D'))) & (df.company == df.company_y), ['company', 'date', 'end_date', 'measure']].groupby(['company', 'end_date']).measure.sum()

In [ ]:
df = pd.DataFrame({'Origin': {1: 'A', 6: 'A', 11: 'A', 16: 'A', 21: 'B', 26: 'B', 31: 'C', 36: 'C'},
 'Destination': {1: 'B', 6: 'B', 11: 'C', 16: 'C', 21: 'Z', 26: 'Z', 31: 'Z', 36: 'Z'}, 
 'Dept_Time': {1: pd.Timestamp('2019-03-30 17:31:00'), 6: pd.Timestamp('2019-05-16 17:32:00'), 11: pd.Timestamp('2019-04-01 08:30:00'), 16: pd.Timestamp('2019-06-09 08:20:00'), 21: pd.Timestamp('2019-07-26 08:31:00'), 26: pd.Timestamp('2019-03-31 06:16:00'), 31: pd.Timestamp('2019-07-03 23:52:00'), 36: pd.Timestamp('2019-03-27 17:31:00')}, 'Arrv_Time': {1: pd.Timestamp('2019-03-30 23:23:00'), 6: pd.Timestamp('2019-05-16 23:22:00'), 11: pd.Timestamp('2019-04-01 14:22:00'), 16: pd.Timestamp('2019-06-09 14:18:00'), 21: pd.Timestamp('2019-07-26 14:23:00'), 26: pd.Timestamp('2019-06-18 05:00:00'), 31: pd.Timestamp('2019-07-04 05:36:00'), 36: pd.Timestamp('2019-03-27 23:23:00')}})
df

In [ ]:
df.columns

In [ ]:
janitor.lt_join?

In [ ]:
df.lt_join(df, 'Dept_Time', 'Arrv_Time').filter(['Dept_Time_x', 'Arrv_Time_y'])

In [ ]:
df1 = pd.DataFrame({'col_a': [1,2,3], 'col_b': ["A", "B", "C"]})
df1

In [ ]:
df2 = pd.DataFrame({'col_a': [0, 2, 3], 'col_c': ["Z", "X", "Y"]})
df2

In [ ]:
df1.gt_join(df2, 'col_a', 'col_a', suffixes=(None, '_y'))

In [ ]:
TableA = pd.DataFrame(np.random.rand(4, 3),
                      pd.Index(list('abcd'), name='Key'),
                      ['A', 'B', 'C']).reset_index()
TableB = pd.DataFrame(np.random.rand(4, 3),
                      pd.Index(list('aecf'), name='Key'),
                      ['A', 'B', 'C']).reset_index()

In [ ]:
TableA

In [ ]:
TableB

In [ ]:
TableA.merge(TableB, on='Key', how='outer', indicator=True).query("_merge ==['left_only', 'right_only']")

In [ ]:
A = dict(x=[1,2,2,2,4,4,4], y=[2,3,4,6,5,5,10], z=[2,1,7,4,5,6,3])
B = dict(x=[2,4],y=[3,8])
A = pd.DataFrame(A)
B = pd.DataFrame(B)

In [ ]:
A

In [ ]:
B

In [ ]:
A=pd.concat([A]*10_000, ignore_index=True)
B= pd.concat([B]*10, ignore_index=True)

In [ ]:
%%timeit
index_to_na = A.le_join(B, "x", "x", suffixes=(None, '_y')).loc[lambda df: df.y > df.y_y, ["x", "y", "z"]]
A.loc[lambda df: df.z.isin(index_to_na.z), 'z'] = pd.NA
A

In [ ]:
A

In [ ]:
left = pd.DataFrame([[1, 2, 3],
                     [4, 5, 6],
                    [7, 8, 9]],columns=['a', 'b', 'c'])


right = pd.DataFrame([[1, 2.5, -3],
                      [4, 7, -6],
                   [6, 8.5, -9]],
                    columns=['a', 'd', 'e'])

left

In [ ]:
right

In [ ]:
left.lt_join(right, 'b', 'd').query('c>d')

In [ ]:
pd.DataFrame([[1, 2, 3, 2.5, -3]],
                     columns=['a', 'b', 'c', 'd', 'e'])

In [ ]:
data = """CHR  SNP  POS
1  rs2073813  753541
1  rs3131969  754182
2  rs3131968  754192
2  rs3131967  754334
3  rs3115859  754503
3  rs3131966  900000

"""

left = pd.read_csv(StringIO(data), sep="\s{2,}", engine='python')

left

In [ ]:
data = """CHR  start  end 
1  700500  833300
2  1000  20000
2  59998  60000 
3  700000  800000

"""

right = pd.read_csv(StringIO(data), sep="\s{2,}", engine='python')

right

In [ ]:
condition1 = lambda df: (df.POS.ge(df.start)) & (df.CHR.eq(df.CHR_y))
left.le_join(right, "POS", "end", suffixes=(None, "_y")).loc[condition1, ["CHR", "SNP", "POS"]]

In [ ]:
df1 = pd.DataFrame(
    {
        'TranDate': ['2018/01/05', '2018/02/07', '2018/03/24']
        ,'Loc': [4000, 5000, 4200]
    }
)

df2 = pd.DataFrame(
    {
        'StartDate': ['2018/01/01', '2018/02/01', '2018/03/01']
        ,'EndDate': ['2018/01/31','2018/02/28', '2018/03/30']
        ,'PP': ['01', '02', '03']
    }
)

In [ ]:
df1

In [ ]:
df2

In [ ]:
dfR = pd.DataFrame(
    {
        'TranDate': ['2018/01/05', '2018/02/07', '2018/03/24']
        ,'Loc': [4000, 5000, 4200]
        ,'PP': ['01', '02', '03']
    }
)

dfR

In [ ]:

df1['TranDate'] = pd.to_datetime(df1['TranDate'])
df1.dtypes

In [ ]:
df2.StartDate = pd.to_datetime(df2.StartDate)
df2.EndDate = pd.to_datetime(df2.EndDate)

In [ ]:
df2.dtypes

In [ ]:
df1.le_join(df2, 'TranDate', 'EndDate').loc[lambda df: df.TranDate.ge(df.StartDate), ["TranDate", "Loc", "PP"]]

In [ ]:
data = """rank    begin    end     labels
first   30953   31131    label1
first   31293   31435    label2
first   31436   31733    label4
first   31734   31754    label1
first   32841   33037    label3
second  33048   33456    label4

"""

left = pd.read_csv(StringIO(data), sep="\s{2,}", engine='python')

left

In [ ]:
data = """rank    start 
first   31333     
first   31434     
first   33039    
first   33123     
first   33125     
"""

right = pd.read_csv(StringIO(data), sep="\s{2,}", engine='python')

right

In [ ]:
left

In [ ]:
right.filter(['rank', 'start', 'labels']).merge(right.le_join(left, "start", "end", suffixes=(None, "_y")).loc[lambda df: df.start.gt(df.begin), ['rank', 'start', 'labels']], how='left')